In [17]:
# For Preprocessing
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from xgboost import plot_importance
from sklearn.metrics import confusion_matrix
import numpy as np
from collections import Counter
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier
import os
from tqdm import tqdm_notebook as tqdm
from sklearn.externals import joblib
%matplotlib inline
import seaborn as sns
import dask.dataframe as dd
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score,recall_score,precision_score
from dask.distributed import Client

In [18]:
X = pd.read_csv("application_train.csv")
df_test = pd.read_csv("application_test.csv")
X.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
X['CODE_GENDER'].replace('XNA',np.nan, inplace=True)
X['CODE_GENDER'].value_counts()

F    202448
M    105059
Name: CODE_GENDER, dtype: int64

In [20]:
X['DAYS_LAST_PHONE_CHANGE'].replace(0, np.nan, inplace=True)

In [21]:
X['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)

In [25]:

le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in X:
    if X[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(X[col].unique())) <= 2:
            # Train on the training data
            le.fit(X[col])
            # Transform both training and testing data
            X[col] = le.transform(X[col])
           # df_test[col] = le.transform(df_test[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
print('%d columns were label encoded.' % le_count)


3 columns were label encoded.


In [26]:
# one-hot encoding of categorical variables
X = pd.get_dummies(X)
#df_test = pd.get_dummies(df_test)
list1=X.columns

print('Training Features shape: ', X.shape)
#print('Testing Features shape: ', df_test.shape)
print(list1)

Training Features shape:  (307511, 242)
Index(['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE', 'FLAG_OWN_CAR',
       'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT',
       'AMT_ANNUITY', 'AMT_GOODS_PRICE',
       ...
       'HOUSETYPE_MODE_terraced house', 'WALLSMATERIAL_MODE_Block',
       'WALLSMATERIAL_MODE_Mixed', 'WALLSMATERIAL_MODE_Monolithic',
       'WALLSMATERIAL_MODE_Others', 'WALLSMATERIAL_MODE_Panel',
       'WALLSMATERIAL_MODE_Stone, brick', 'WALLSMATERIAL_MODE_Wooden',
       'EMERGENCYSTATE_MODE_No', 'EMERGENCYSTATE_MODE_Yes'],
      dtype='object', length=242)


In [27]:
# Function to calculate missing values by column# Funct 
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [28]:
# Missing values statistics
missing_values = missing_values_table(X)
mis_col=list(missing_values.index)
Mis=X[mis_col][:]

for col in mis_col:
    X[col+'_miss']=[0]*len(X[col])
    X[col+'_miss'][list(X[np.isnan(X[col])].index)]=1

print(X.describe())

Your selected dataframe has 242 columns.
There are 62 columns that have missing values.


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


          SK_ID_CURR         TARGET  NAME_CONTRACT_TYPE   FLAG_OWN_CAR  \
count  307511.000000  307511.000000       307511.000000  307511.000000   
mean   278180.518577       0.080729            0.095213       0.340108   
std    102790.175348       0.272419            0.293509       0.473746   
min    100002.000000       0.000000            0.000000       0.000000   
25%    189145.500000       0.000000            0.000000       0.000000   
50%    278202.000000       0.000000            0.000000       0.000000   
75%    367142.500000       0.000000            0.000000       1.000000   
max    456255.000000       1.000000            1.000000       1.000000   

       FLAG_OWN_REALTY   CNT_CHILDREN  AMT_INCOME_TOTAL    AMT_CREDIT  \
count    307511.000000  307511.000000      3.075110e+05  3.075110e+05   
mean          0.693673       0.417052      1.687979e+05  5.990260e+05   
std           0.460968       0.722121      2.371231e+05  4.024908e+05   
min           0.000000       0.000000    

In [29]:
# Shashank imputer
from sklearn.base import TransformerMixin

class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)
    
print('before...')    
Mis=X[mis_col][:]
print('Mis - ',Mis)

xt = DataFrameImputer().fit_transform(Mis)

print('after...')
print(xt)

before...
Mis -          COMMONAREA_MODE  COMMONAREA_AVG  COMMONAREA_MEDI  \
0                0.0144          0.0143           0.0144   
1                0.0497          0.0605           0.0608   
2                   NaN             NaN              NaN   
3                   NaN             NaN              NaN   
4                   NaN             NaN              NaN   
5                   NaN             NaN              NaN   
6                   NaN             NaN              NaN   
7                   NaN             NaN              NaN   
8                   NaN             NaN              NaN   
9                   NaN             NaN              NaN   
10                  NaN             NaN              NaN   
11                  NaN             NaN              NaN   
12                  NaN             NaN              NaN   
13               0.0587          0.0582           0.0585   
14               0.1153          0.1143           0.1150   
15                  NaN

[307511 rows x 62 columns]
after...
        COMMONAREA_MODE  COMMONAREA_AVG  COMMONAREA_MEDI  \
0              0.014400        0.014300         0.014400   
1              0.049700        0.060500         0.060800   
2              0.042553        0.044621         0.044595   
3              0.042553        0.044621         0.044595   
4              0.042553        0.044621         0.044595   
5              0.042553        0.044621         0.044595   
6              0.042553        0.044621         0.044595   
7              0.042553        0.044621         0.044595   
8              0.042553        0.044621         0.044595   
9              0.042553        0.044621         0.044595   
10             0.042553        0.044621         0.044595   
11             0.042553        0.044621         0.044595   
12             0.042553        0.044621         0.044595   
13             0.058700        0.058200         0.058500   
14             0.115300        0.114300         0.115000   
15  

[307511 rows x 62 columns]


In [30]:
# # #Have commented the dropping (not dropping any columns for now) Replace missing with -999
# # Mis=X[mis_col][:]
# # X_mis=(Mis.isnull().sum() / Mis.shape[0] * 100.00)
# # df_mis=pd.DataFrame(np.c_[X_mis.index,X_mis],columns=['Colname','miss_p'])
# # drop_col=df_mis[df_mis['miss_p']>65]['Colname'].values
# # print('number of columns before dropping->',len(X.columns))
# # #X=X.drop(drop_col,axis=1)
# # print('number of columns after dropping->',len(X.columns))
# X.replace(np.nan,-999, inplace=True)
# #X.astype('float32',inplace=True)
# #print(newX[].describe())
# y=X['TARGET']
# X=X.drop('TARGET',axis=1)
for mis in mis_col:
    X[mis]=xt[mis]
print(X.shape)
print(X.describe())

(307511, 304)
          SK_ID_CURR         TARGET  NAME_CONTRACT_TYPE   FLAG_OWN_CAR  \
count  307511.000000  307511.000000       307511.000000  307511.000000   
mean   278180.518577       0.080729            0.095213       0.340108   
std    102790.175348       0.272419            0.293509       0.473746   
min    100002.000000       0.000000            0.000000       0.000000   
25%    189145.500000       0.000000            0.000000       0.000000   
50%    278202.000000       0.000000            0.000000       0.000000   
75%    367142.500000       0.000000            0.000000       1.000000   
max    456255.000000       1.000000            1.000000       1.000000   

       FLAG_OWN_REALTY   CNT_CHILDREN  AMT_INCOME_TOTAL    AMT_CREDIT  \
count    307511.000000  307511.000000      3.075110e+05  3.075110e+05   
mean          0.693673       0.417052      1.687979e+05  5.990260e+05   
std           0.460968       0.722121      2.371231e+05  4.024908e+05   
min           0.000000     

In [31]:
# iterate below functions only

In [32]:
y=X['TARGET']
X=X.drop('TARGET',axis=1)

def split_train_test(X , y):
    return(train_test_split( X , y , test_size=0.2, random_state=42))

X_train, X_test, y_train, y_test =split_train_test(X,y)

In [33]:
def feature_eng(X_train,y_train,X_test,y_test):
    train_col=list(X_train.columns)
    test_col=list(X_test.columns)    
    clf = ExtraTreesClassifier(n_estimators=50)
    clf = clf.fit(X_train, y_train)
    model = SelectFromModel(clf, prefit=True)
    new_trainX = model.transform(X_train)
    n=np.shape(new_trainX)[1]
    importances = clf.feature_importances_
    indices = np.argsort(importances)[::-1]
    clist=[]
    for f in range(n):     
        clist.append(indices[f])
    new_train_col=[train_col[i] for i in clist]
    new_test_col=[test_col[i] for i in clist]
    X_train=pd.DataFrame(X_train)
    X_test=pd.DataFrame(X_test)
    y_train=pd.DataFrame(y_train)
    y_test=pd.DataFrame(y_test)
    df_new_train=pd.DataFrame(X_train.loc[:,new_train_col],columns=new_train_col)            
    df_new_test=pd.DataFrame(X_test.loc[:,new_test_col],columns=new_test_col)
    print("Improved number of features->",len(clist))    
    return(df_new_train,df_new_test)

df_new_train,df_new_test=feature_eng(X_train, y_train, X_test, y_test)

Improved number of features-> 98


In [34]:
def run_model_eval_metric(modelname,modelobj,X_train,Y_train,X_test,Y_test):
    # In order to let the models run faster we use dask dataframe
    
    modelobj.fit(X_train,Y_train)
    ypred_train=modelobj.predict(X_train)
    ypred_test=modelobj.predict(X_test)
    y_train_proba = modelobj.predict_proba(X_train)[:,1]
    y_test_proba = modelobj.predict_proba(X_test)[:,1]
    print('\n')
    print(modelname,"model:")
    confmat=confusion_matrix(Y_train,ypred_train)
    print("Train Confusion Matrix:")
    print(confmat)
    confmat=confusion_matrix(Y_test,ypred_test)
    print("Test Confusion Matrix:")
    print(confmat)
    metric=pd.DataFrame([[0]*5]*2,columns=['PRECISION','RECALL','F1_SCORE','ROC_AUC_SCORE','ACCURACY'],index=['Train','Test'],dtype='float32')
    metric['PRECISION']['Train']=round(precision_score(Y_train, ypred_train),3)
    metric['RECALL']['Train']=recall_score(Y_train, ypred_train)
    metric['F1_SCORE']['Train']=round(f1_score(Y_train, ypred_train),3)
    metric['ROC_AUC_SCORE']['Train']=roc_auc_score(Y_train,y_train_proba)
    metric['ACCURACY']['Train']=accuracy_score(Y_train,ypred_train)
    metric['PRECISION']['Test']=precision_score(Y_test, ypred_test)
    metric['RECALL']['Test']=recall_score(Y_test, ypred_test)
    metric['F1_SCORE']['Test']=f1_score(Y_test, ypred_test)
    metric['ROC_AUC_SCORE']['Test']=roc_auc_score(Y_test,y_test_proba)
    metric['ACCURACY']['Test']=accuracy_score(Y_test,ypred_test)
    print(metric)
   
    

In [35]:
def run_all_models(df_new_train,y_train,df_new_test,y_test):
    # K Nearest Neighbor
    knearest = KNeighborsClassifier(algorithm= 'auto', n_jobs= -1, n_neighbors= 1)
    run_model_eval_metric('K-Nearest Neighbor 1',knearest,df_new_train,y_train,df_new_test,y_test)
    
    knearest = KNeighborsClassifier(algorithm= 'auto', n_jobs= -1, n_neighbors= 3)
    run_model_eval_metric('K-Nearest Neighbor 3',knearest,df_new_train,y_train,df_new_test,y_test)

    # Random Forest
    randfor = RandomForestClassifier(n_estimators= 5, max_features= 'auto', max_depth= 60, bootstrap= True, n_jobs= -1)
    run_model_eval_metric('Random Forest',randfor,df_new_train,y_train,df_new_test,y_test)

    # Decision Trees
    dectree = DecisionTreeClassifier(random_state= 42, max_features= 'auto', max_depth= 40, criterion= 'gini')
    run_model_eval_metric('Decision Tree',dectree,df_new_train,y_train,df_new_test,y_test)

    # Logistic Regression
    logreg = LogisticRegression(penalty = 'l1', class_weight= {0: 0.1, 1: 0.9}, C= 0.5)
    run_model_eval_metric('Logistic Regression',logreg,df_new_train,y_train,df_new_test,y_test)

run_all_models(df_new_train,y_train,df_new_test,y_test)




K-Nearest Neighbor 1 model:
Train Confusion Matrix:
[[226132      0]
 [     0  19876]]
Test Confusion Matrix:
[[52026  4528]
 [ 4413   536]]
       PRECISION    RECALL  F1_SCORE  ROC_AUC_SCORE  ACCURACY
Train   1.000000  1.000000  1.000000        1.00000  1.000000
Test    0.105845  0.108305  0.107061        0.51412  0.854625


K-Nearest Neighbor 3 model:
Train Confusion Matrix:
[[224307   1825]
 [ 16033   3843]]
Test Confusion Matrix:
[[55363  1191]
 [ 4784   165]]
       PRECISION    RECALL  F1_SCORE  ROC_AUC_SCORE  ACCURACY
Train   0.678000  0.193349  0.301000       0.934129  0.927409
Test    0.121681  0.033340  0.052339       0.527903  0.902850


Random Forest model:
Train Confusion Matrix:
[[225902    230]
 [  3629  16247]]
Test Confusion Matrix:
[[55749   805]
 [ 4692   257]]
       PRECISION    RECALL  F1_SCORE  ROC_AUC_SCORE  ACCURACY
Train   0.986000  0.817418   0.89400       0.994146  0.984314
Test    0.241996  0.051930   0.08551       0.600245  0.910622


Decision Tree mode

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)




Logistic Regression model:
Train Confusion Matrix:
[[173176  52956]
 [  8100  11776]]
Test Confusion Matrix:
[[43215 13339]
 [ 2001  2948]]
       PRECISION    RECALL  F1_SCORE  ROC_AUC_SCORE  ACCURACY
Train   0.182000  0.592473  0.278000       0.747326  0.751813
Test    0.181003  0.595676  0.277642       0.748306  0.750581
